In [1]:
import sklearn
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import os
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
cd "/content/drive/MyDrive/money_data"

/content/drive/MyDrive/money_data


In [4]:
ls

100000dong/  10000dong/  200000dong/  20000dong/  50000dong/


In [5]:
# preprocessing
link = "/content/drive/MyDrive/money_data"
money_names = os.listdir(link)
money_path = []
for money_name in money_names:
  money_path.append(os.path.join(link, money_name))
print(money_names)
X = []
y = []
for i in range(len(money_path)):
  print(money_names[i])
  img_path = list(Path(money_path[i]).glob("*"))
  for j in img_path:
    img = plt.imread(j)
    img = cv2.resize(img, (128, 128))
    img.astype('float32')
    img = img/255
    X.append(img)
    y.append(i)

['10000dong', '20000dong', '50000dong', '100000dong', '200000dong']
10000dong
20000dong
50000dong
100000dong
200000dong


In [6]:
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)

(954, 128, 128, 3)
(954,)


In [7]:
from keras.utils import to_categorical
y = to_categorical(y)
print(y.shape)

(954, 5)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
from keras import models, layers
from keras.layers import Flatten, Dense, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils 
model = models.Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(128,128,3),padding = 'same'))
model.add(MaxPooling2D((2,2),padding='same'))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu',padding = 'same'))
model.add(MaxPooling2D((2,2),padding='same'))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu',padding = 'same'))
model.add(MaxPooling2D((2,2),padding='same'))
model.add(Flatten(input_shape=[128, 128, 3]))
model.add(Dense(2000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(3000, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(money_names), activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 128)      0

In [10]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, epochs=300, batch_size=128)

Epoch 1/300
6/6 [==============================] - 6s 306ms/step - loss: 8.4801 - accuracy: 0.3421
Epoch 2/300
6/6 [==============================] - 1s 106ms/step - loss: 1.2221 - accuracy: 0.3840
Epoch 3/300
6/6 [==============================] - 1s 106ms/step - loss: 1.0708 - accuracy: 0.4168
Epoch 4/300
6/6 [==============================] - 1s 105ms/step - loss: 1.0700 - accuracy: 0.4010
Epoch 5/300
6/6 [==============================] - 1s 106ms/step - loss: 1.0645 - accuracy: 0.4010
Epoch 6/300
6/6 [==============================] - 1s 105ms/step - loss: 1.0708 - accuracy: 0.3971
Epoch 7/300
6/6 [==============================] - 1s 108ms/step - loss: 1.0649 - accuracy: 0.4142
Epoch 8/300
6/6 [==============================] - 1s 107ms/step - loss: 1.0619 - accuracy: 0.4168
Epoch 9/300
6/6 [==============================] - 1s 106ms/step - loss: 1.0595 - accuracy: 0.4050
Epoch 10/300
6/6 [==============================] - 1s 106ms/step - loss: 1.1298 - accuracy: 0.3801
Epoch 11/

In [12]:
n = 123
print(f'{n}/{len(y_test)}')
pred = model.predict(X_test[n].reshape((1, 128, 128, 3)))
print(f'Predict: {money_names[pred.argmax()]}  ({int(pred[0][pred.argmax()]*100)}%)')
print('True_label: ', money_names[y_test[n].argmax()])

123/191
1/1 [==============================] - 0s 202ms/step
Predict: 20000dong  (100%)
True_label:  20000dong
